# Fasttext experiments with tactus data

First import required modules: general Python modules and modules from Orange3

In [ ]:
import csv
import fasttext
import gzip
import os
import re
import sys

sys.path.append("/home/erikt/projects/e-mental-health/enron/orange-hackathon/orangehackathon/libs")
import tactusloaderLIB
import OWEmailSorterLIB
import markduplicatesLIB
import removemarkedtextLIB
import LIWCLIB

Next, read the mails from the data set via Orange 3 and store them in the data structure allMails. This takes several minutes so we do not want to do this often.

In [ ]:
ANODIRECTORY = "/home/erikt/projects/e-mental-health/usb/releases/20191217/"
ANOSTRING = "-an"
GZEXTENSION = ".gz"
XMLEXTENSION = ".xml"

os.chdir(ANODIRECTORY)

def shortenFileName(fileName):
    fileName = re.sub(GZEXTENSION,"",fileName)
    fileName = re.sub(XMLEXTENSION,"",fileName)
    fileName = re.sub(ANOSTRING,"",fileName)   
    return(fileName)

In [ ]:
LASTFILENBR = 1987

def getAllTactusMails():
    allMails = {}
    missingFiles = []
    for patientId in list(range(1,LASTFILENBR+1)):
        if patientId % 100 == 0: print(patientId,end=" ")
        fileName = tactusloaderLIB.makeFileName(str(patientId))+GZEXTENSION
        if os.path.isfile(ANODIRECTORY+fileName):
            mails = tactusloaderLIB.processFile(ANODIRECTORY,fileName)
            if len(mails[0]) > 0:
                sortedMails = OWEmailSorterLIB.filterEmails(mails[0],filter_asc=True)
                markedMails = markduplicatesLIB.processCorpus(sortedMails)
                strippedMails = removemarkedtextLIB.processCorpus(markedMails)
                allMails[shortenFileName(fileName)] = strippedMails
        else: missingFiles.append(fileName)
    if len(missingFiles) > 0: print("\nmissing files:",missingFiles)
    return(allMails)

allMails = getAllTactusMails()

We also need labels for the data. We will use the dropout labels provided by a student's project

In [ ]:
SELECTEDFILE = "/home/erikt/projects/e-mental-health/usb/releases/20200305/selected.csv.gz"
DROPOUT = "dropout"
FILE = "file"
NBROFDROPOUTS = 791

dropouts = {}
inFile = gzip.open(SELECTEDFILE,"rt",encoding="utf-8")
csvreader = csv.DictReader(inFile)
for row in csvreader:
    row[FILE] = re.sub("(-an)?.xml(.gz)?$","",row[FILE])
    if row[DROPOUT] == "1" or row[DROPOUT] == "2": 
        dropouts[row[FILE]] = row[DROPOUT]
inFile.close()

len(dropouts) == NBROFDROPOUTS

Fasttext operates on files so we should store our data in a file to enable fasttext to access it.

In [ ]:
import random

COUNSELOR = "COUNSELOR"
FROMFIELD = "from"
LABELPREFIX = "__label__"
OUTFILENAME = "fasttext.txt"

def getFieldId(corpus,fieldName):
    fieldId = -1
    for i in range(0,len(corpus.domain.metas)):
        if str(corpus.domain.metas[i]) == fieldName:
            fieldId = i
    return(fieldId)

def getLastCounselorMailId(allMails,clientId):
    for i in range(-1,-len(allMails[clientId])-1,-1):
        if allMails[clientId][i][FROMFIELD] == COUNSELOR: return(i)
    sys.exit("getLastCounselortMailId: client "+clientId+" did not receive any emails!")

def selectLastCounselorMail(allMails):
    selectedData = []
    selectedLabels = []
    if len(allMails) > 0:
        firstClient = list(allMails.keys())[0]
        subjectId = getFieldId(allMails[firstClient],"subject")
        textId = getFieldId(allMails[firstClient],"text")
        for clientId in allMails:
            if clientId in dropouts:
                lastCounselorMailId = getLastCounselorMailId(allMails,clientId)
                subject = allMails[clientId][lastCounselorMailId].metas[subjectId]
                mailText = allMails[clientId][lastCounselorMailId].metas[textId]
                selectedLabels.append(dropouts[clientId])
                selectedData.append(subject+" "+mailText)
    return(selectedData,selectedLabels)

def storeTactusDataInFastTextFile(X,y,outFileName):
    if len(X) != len(y): 
        sys.exit("storeTactusDataInFastTextFile(): incompatable lengths of X and y")
    outFile = open(outFileName,"w")
    for i in range(0,len(X)):
        print(LABELPREFIX+str(y[i]),X[i],file=outFile)
    outFile.close()
    return()

def shuffleXy(X,y):
    if len(X) != len(y): sys.exit("shuffleXy(): incompatable lengths of X and y")
    shuffledX = []
    shuffledY = []
    while len(X) > 0:
        r = random.randint(0,len(X)-1)
        shuffledX.append(X[r])
        shuffledY.append(y[r])
        X[r] = X[0]
        y[r] = y[0]
        X.pop(0)
        y.pop(0)
    return(shuffledX,shuffledY)
    
selectedData,selectedLabels = selectLastCounselorMail(allMails)
X,y = shuffleXy(selectedData,selectedLabels)

Now we can start a fasttext experiment using this data set

In [ ]:
NBROFFOLDS = 5
foldBoundaries = [round((f)*len(X)/NBROFFOLDS) for f in range(0,NBROFFOLDS+1)]

In [ ]:
DIM = 300
EPOCHSTART = 30
EPOCHEND = 31
EPOCHSTEP = 5
TRAINFILE = "fasttext-train.txt"
TESTFILE = "fasttext-test.txt"
WIKIFILENAME = "wiki.nl.vec"
CCFILENAME = "cc.nl.300.vec"
WIKIDIR = "/home/erikt/projects/newsgac/fasttext-runs/"

def fasttextPredict(X,y,epochStart,epochEnd,epochStep):
    predictions = {str(epoch):[] for epoch in range(EPOCHSTART,EPOCHEND,EPOCHSTEP)}
    for f in range(0,NBROFFOLDS):
        startTest = foldBoundaries[f]
        endTest = foldBoundaries[f+1]
        storeTactusDataInFastTextFile(X[startTest:endTest],y[startTest:endTest],TESTFILE)
        storeTactusDataInFastTextFile(X[:startTest]+X[endTest:],y[:startTest]+y[endTest:],TRAINFILE)
        for epoch in range(epochStart,epochEnd,epochStep):
            model = fasttext.train_supervised(ANODIRECTORY+TRAINFILE,dim=DIM,epoch=epoch) 
                                            # pretrainedVectors=WIKIDIR+WIKIFILENAME)
            testFile = open(TESTFILE,"r")
            for line in testFile:
                tokens = line.strip().split()
                if re.search("^"+LABELPREFIX,tokens[0]): tokens.pop(0)
                line = " ".join(tokens)
                predictions[str(epoch)].append(re.sub(LABELPREFIX,"",model.predict(line)[0][0]))
            testFile.close()
    return(predictions)

predictions = fasttextPredict(X,y,EPOCHSTART,EPOCHEND,EPOCHSTEP)

In [ ]:
import numpy as np

uniqueCounts = np.unique(y,return_counts=True)
totals = {list(uniqueCounts[0])[i]:list(uniqueCounts[1])[i] for i in range(0,len(uniqueCounts[0]))}
totals['0'] = len(y)

for epoch in predictions:
    counts = {'0':0,'1':0,'2':0}
    for i in range(0,len(predictions[epoch])):
        if predictions[epoch][i] == y[i]: 
            counts['0'] += 1
            counts[y[i]] += 1
    print(epoch,end = " # ")
    for key in counts: print(key,":",round(counts[key]/totals[key],3),end="; ",sep="")
    print()

When trying to predict dropout based on the final counselor mail, using a Wikipedia dictionary did not improve accuracy (83.7% vs 85.2%), at least not when training 30 epochs.

## Model explanation

Find out which tokens contribute to which classes.

In [ ]:
storeTactusDataInFastTextFile(X,y,TRAINFILE)
model = fasttext.train_supervised(ANODIRECTORY+TRAINFILE,dim=DIM,epoch=EPOCHSTART)

seen = {}
predictions = {}
for mailText in X:
    for token in mailText.split():
        if not token in seen:
            seen[token] = True
            labels,scores = model.predict(token)
            for i in range(0,len(labels)):
                label = list(labels)[i]
                score = list(scores)[i]
                if not label in predictions: predictions[label] = {}
                predictions[label][token] = score

In [ ]:
len(predictions["__label__1"]),len(predictions["__label__2"])

In [ ]:
sorted1 = {k: v for k, v in sorted(predictions["__label__1"].items(), key=lambda item: item[1], reverse=True)}
{ key:sorted1[key] for key in list(sorted1.keys())[0:20] }

In [ ]:
sorted2 = {k: v for k, v in sorted(predictions["__label__2"].items(), key=lambda item: item[1], reverse=True)}
{ key:sorted2[key] for key in list(sorted2.keys())[0:20] }

There are many more tokens that trigger the largest class (1/dropout) rather than the smallest (2/finisher; about 3200 vs about 80). The tokens with the highest scores according to fastText do not seem very interesting.